<a href="https://colab.research.google.com/github/jellio9/Absorption_with_lvlspy/blob/develop/Absorption_with_lvlspy_JacobElliott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Absorption with lvlspy - Jacob Elliott


The goal of this notebook is to display the absorption spectrum from an optically thin gas of atoms. We will assume a flat spectrum for the incident radiation over a certain range, and we will look at the spectrum as a function of temperature of the gas.

I will begin by installing lvlspy and other necessary libraries:

In [ ]:
import sys, subprocess, importlib.util
required = {'numpy','lvlspy','matplotlib','scipy'}
installed = {pkg for pkg in required if importlib.util.find_spec(pkg) is not None}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])

import numpy as np
import lvlspy.level as lv
import lvlspy.spcoll as lc
import lvlspy.species as ls
import lvlspy.transition as lt
import lvlspy.io as li
from lvlspy.io import xml

from ipywidgets import interact, interactive
import ipywidgets as widgets
from scipy.integrate import odeint


!{sys.executable} -m pip install --quiet matplotlib

import matplotlib.pyplot as plt

#From the Example notebook:

Define some routines for the notebook.  Create an array.

In [ ]:
x = np.linspace(0, 1)

Now plot a line with variable slope.

In [ ]:
def plot_with_slope(m):
    plt.plot(x, m * x, label="m = {:.1f}".format(m))
    plt.xlabel("x")
    plt.ylabel("y = m * x")
    plt.legend()

interact(plot_with_slope, m = widgets.BoundedFloatText(value=1, min=0, max = 10, step=0.1,
                                                       descriptions='Slope:', disabled=False))